In [1]:
!pip install -U wandb huggingface_hub
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.0/515.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.2 which is incompatible.
Looking in indexes: https://download.pytorch.org/whl/cu121


# Q1


In [2]:
#@title 1) Imports, Args, Repro
import os, zipfile, argparse, random, math, json
from pathlib import Path

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from torchvision.models import resnet18, resnet34, resnet50
from torchvision.datasets.folder import default_loader

import numpy as np
import wandb
from PIL import Image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--project", type=str, default="tiny-imagenet-resnet")
    p.add_argument("--entity", type=str, default=None)
    p.add_argument("--epochs", type=int, default=8)
    p.add_argument("--batch_size", type=int, default=128)
    p.add_argument("--lr", type=float, default=0.01)
    p.add_argument("--model", type=str, default="resnet18", choices=["resnet18","resnet34","resnet50"])
    p.add_argument("--data_dir", type=str, default="./data/tiny-imagenet-200")
    p.add_argument("--num_workers", type=int, default=4)
    p.add_argument("--seed", type=int, default=42)
    return p.parse_args(args=[])

ARGS = parse_args()

In [3]:
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

In [4]:
set_seed(ARGS.seed)
print("Device:", DEVICE)

Device: cuda


In [5]:
#@title 2) Download & Extract Tiny ImageNet-200
DATA_ROOT = Path(ARGS.data_dir)
DATA_ROOT.parent.mkdir(parents=True, exist_ok=True)
zip_path = DATA_ROOT.parent / "tiny-imagenet-200.zip"

if not DATA_ROOT.exists():
    if not zip_path.exists():
        import urllib.request
        url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
        print("Downloading Tiny ImageNet (~237MB)...")
        urllib.request.urlretrieve(url, zip_path)
    print("Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(DATA_ROOT.parent)
    print("Done.")
else:
    print("Tiny-ImageNet already available:", DATA_ROOT)


Extracting...
Done.


In [6]:
#@title 3) Datasets & Dataloaders (ImageNet normalization, resize to 224)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

VAL_DIR = DATA_ROOT / "val"
VAL_IMAGES_DIR = VAL_DIR / "images"
VAL_ANNO = VAL_DIR / "val_annotations.txt"
VAL_PREP_DIR = DATA_ROOT / "val_prepared"

def prepare_val_split():
    if VAL_PREP_DIR.exists():
        return
    VAL_PREP_DIR.mkdir(parents=True, exist_ok=True)
    mapping = {}
    with open(VAL_ANNO, "r") as f:
        for line in f:
            img, cls, *_ = line.strip().split("\t")
            mapping[img] = cls
    for img_name, cls in mapping.items():
        cls_dir = VAL_PREP_DIR / cls
        cls_dir.mkdir(parents=True, exist_ok=True)
        src = VAL_IMAGES_DIR / img_name
        dst = cls_dir / img_name
        if not dst.exists():
            try:
                os.link(src, dst)
            except Exception:
                from shutil import copy2
                copy2(src, dst)

prepare_val_split()

train_dir = DATA_ROOT / "train"
val_dir   = VAL_PREP_DIR

train_ds = torchvision.datasets.ImageFolder(train_dir, transform=train_tfms)
val_ds   = torchvision.datasets.ImageFolder(val_dir,   transform=val_tfms)
NUM_CLASSES = 200

train_loader = DataLoader(train_ds, batch_size=ARGS.batch_size, shuffle=True,  num_workers=ARGS.num_workers, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=ARGS.batch_size, shuffle=False, num_workers=ARGS.num_workers, pin_memory=True)

print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Classes: {NUM_CLASSES}")


Train: 100000 | Val: 10000 | Classes: 200


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [7]:
#@title 4) Authenticate & Init W&B
print("Follow the prompt to authenticate W&B.")
wandb.login()
run = wandb.init(project=ARGS.project, entity=ARGS.entity, config=vars(ARGS))


Follow the prompt to authenticate W&B.


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 142502005 (142502005-iit-palakkad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
#@title 5) Build Model, Optimizer, Sched
def build_model(name="resnet18", num_classes=200, pretrained=True):
    if name == "resnet18":
        m = resnet18(weights="IMAGENET1K_V1" if pretrained else None)
    elif name == "resnet34":
        m = resnet34(weights="IMAGENET1K_V1" if pretrained else None)
    else:
        m = resnet50(weights="IMAGENET1K_V1" if pretrained else None)
    in_feats = m.fc.in_features
    m.fc = nn.Linear(in_feats, num_classes)
    return m

model = build_model(ARGS.model, NUM_CLASSES, pretrained=True).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=ARGS.lr, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=ARGS.epochs)
wandb.watch(model, log="all", log_freq=50)

def acc_top1(logits, y):
    return (logits.argmax(1) == y).float().mean().item()



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 77.5MB/s]


In [9]:
#@title 6) Train + Validate (log curves; save best as artifact)
ckpt_dir = Path("./checkpoints"); ckpt_dir.mkdir(exist_ok=True)
best, best_path = 0.0, ckpt_dir / "best.pt"
global_step = 0

for epoch in range(1, ARGS.epochs+1):
    model.train()
    tr_loss = tr_acc = 0.0
    for i, (x,y) in enumerate(train_loader):
        x,y = x.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits = model(x); loss = criterion(logits, y)
        loss.backward(); optimizer.step()

        a = acc_top1(logits, y)
        tr_loss += loss.item() * x.size(0); tr_acc += a * x.size(0)

        if (i+1) % 20 == 0:
            wandb.log({"train/loss": loss.item(), "train/acc": a, "lr": optimizer.param_groups[0]["lr"], "step": global_step})
        global_step += 1
    scheduler.step()
    tr_loss /= len(train_ds); tr_acc /= len(train_ds)

    model.eval()
    va_loss = va_acc = 0.0
    with torch.no_grad():
        for x,y in val_loader:
            x,y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x); loss = criterion(logits, y)
            va_loss += loss.item() * x.size(0)
            va_acc  += acc_top1(logits, y) * x.size(0)
    va_loss /= len(val_ds); va_acc /= len(val_ds)

    wandb.log({"epoch": epoch, "train/epoch_loss": tr_loss, "train/epoch_acc": tr_acc, "val/loss": va_loss, "val/acc": va_acc})
    print(f"Epoch {epoch:02d}: train_acc={tr_acc:.3f} val_acc={va_acc:.3f}")

    if va_acc > best:
        best = va_acc
        torch.save({"model": model.state_dict(), "val_acc": va_acc, "epoch": epoch, "args": vars(ARGS)}, best_path)
        art = wandb.Artifact(f"{ARGS.model}-tinyimagenet", type="model", metadata={"val_acc": va_acc, **vars(ARGS)})
        art.add_file(str(best_path))
        wandb.log_artifact(art)

print("Best val acc:", best)
run.finish()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 01: train_acc=0.516 val_acc=0.611
Epoch 02: train_acc=0.660 val_acc=0.634
Epoch 03: train_acc=0.708 val_acc=0.667
Epoch 04: train_acc=0.757 val_acc=0.686
Epoch 05: train_acc=0.800 val_acc=0.708
Epoch 06: train_acc=0.841 val_acc=0.719
Epoch 07: train_acc=0.873 val_acc=0.735
Epoch 08: train_acc=0.888 val_acc=0.737
Best val acc: 0.7372


epoch,▁▂▃▄▅▆▇█
lr,███████████▇▇▇▇▇▆▆▆▆▆▆▆▆▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇███
train/acc,▁▂▃▄▅▅▅▆▅▄▄▅▆▆▆▆▆▇▆▇▆▆▇▆▇▇▇▇▆█████▇▇███▇
train/epoch_acc,▁▄▅▆▆▇██
train/epoch_loss,█▅▄▃▂▂▁▁
train/loss,█▅▅▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/acc,▁▂▄▅▆▇██
val/loss,█▇▅▄▃▂▁▁
epoch,8
lr,0.00038


# Q2


In [28]:
#@title 7) Login & Push model weights to HF Hub (Model Repo)
from huggingface_hub import login, HfApi
from pathlib import Path
import json

print("Login to the Hugging Face Hub (use a token with write access).")
login()




Login to the Hugging Face Hub (use a token with write access).


In [19]:
api = HfApi()
user = api.whoami()["name"]
repo_name = f"{ARGS.model}-tinyimagenet-200"
repo_id = f"{user}/{repo_name}"
api.create_repo(repo_id, repo_type="model", exist_ok=True)



RepoUrl('https://huggingface.co/142502005-Anshika/resnet18-tinyimagenet-200', endpoint='https://huggingface.co', repo_type='model', repo_id='142502005-Anshika/resnet18-tinyimagenet-200')

In [20]:
# Save labels + README and upload best checkpoint
idx_to_class = {v:k for k,v in train_ds.class_to_idx.items()}
Path("labels.json").write_text(json.dumps(idx_to_class, indent=2))

readme = f"""---
license: apache-2.0
library_name: pytorch
pipeline_tag: image-classification
tags:
- resnet
- tiny-imagenet
---

# {repo_name}

Pretrained **{ARGS.model}** fine-tuned on Tiny ImageNet-200 (200 classes, input 224x224, ImageNet normalization).
"""
Path("README.md").write_text(readme)

# Upload selected files
api.upload_folder(
    repo_id=repo_id,
    folder_path=".",
    path_in_repo=".",
    allow_patterns=["checkpoints/best.pt", "labels.json", "README.md"],
)
print("Pushed model repo:", repo_id)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  /content/checkpoints/best.pt:  74%|#######4  | 33.5MB / 45.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Pushed model repo: 142502005-Anshika/resnet18-tinyimagenet-200


In [25]:
#@title 8) (Optional) Create a Gradio Space for Inference
from huggingface_hub import HfApi
api = HfApi()

space_name = f"{ARGS.model}-tinyimagenet-demo"
space_id = f"{user}/{space_name}"
api.create_repo(space_id, repo_type="space", space_sdk="gradio", exist_ok=True)




RepoUrl('https://huggingface.co/spaces/142502005-Anshika/resnet18-tinyimagenet-demo', endpoint='https://huggingface.co', repo_type='space', repo_id='142502005-Anshika/resnet18-tinyimagenet-demo')

In [26]:
app_py = f"""
import gradio as gr
import torch
from torchvision import transforms
from torchvision.models import {ARGS.model} as _resnet
from PIL import Image
import requests, json

IMAGENET_MEAN=(0.485,0.456,0.406); IMAGENET_STD=(0.229,0.224,0.225)

def load_model():
    m = _resnet(weights=None)
    m.fc = torch.nn.Linear(m.fc.in_features, 200)
    state = torch.hub.load_state_dict_from_url("https://huggingface.co/{repo_id}/resolve/main/checkpoints/best.pt?download=true", map_location="cpu")
    m.load_state_dict(state["model"], strict=True)
    m.eval()
    return m

model = load_model()

def predict(img: Image.Image):
    tfm = transforms.Compose([
        transforms.Resize(256), transforms.CenterCrop(224),
        transforms.ToTensor(), transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ])
    x = tfm(img).unsqueeze(0)
    with torch.no_grad():
        p = torch.softmax(model(x), dim=1)[0]
    labels = requests.get("https://huggingface.co/{repo_id}/resolve/main/labels.json").json()
    idx_to_class = {{int(k):v for k,v in labels.items()}}
    topk = torch.topk(p, k=5)
    return {{ idx_to_class[int(i)]: float(p[i]) for i in topk.indices }}

demo = gr.Interface(fn=predict, inputs=gr.Image(type="pil"), outputs=gr.Label(num_top_classes=5),
                    title="{space_name}", examples=None)
demo.launch()
"""




In [29]:
req_txt = "torch\ntorchvision\nPillow\ngradio\n"

Path("app.py").write_text(app_py)
Path("requirements.txt").write_text(req_txt)

api.upload_folder(repo_id=space_id, repo_type="space",folder_path=".", path_in_repo=".", allow_patterns=["app.py","requirements.txt"])
print("Space pushed:", space_id)

Space pushed: 142502005-Anshika/resnet18-tinyimagenet-demo


# Q3

In [30]:
#@title 9) Drift utils (brightness/noise) + Eval
from PIL import ImageEnhance
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets.folder import default_loader

def brightness_shift(img: Image.Image, factor: float = 0.4):
    enh = ImageEnhance.Brightness(img)
    return enh.enhance(factor)

def add_gaussian_noise(t: torch.Tensor, sigma: float = 0.15):
    noise = torch.randn_like(t) * sigma
    t = torch.clamp(t + noise, 0.0, 1.0)
    return t

class DriftedValDataset(Dataset):
    def __init__(self, base_dataset, mode="brightness", factor=0.4, sigma=0.15):
        self.base = base_dataset
        self.mode = mode; self.factor = factor; self.sigma = sigma
        self.pre = transforms.Resize(256)
        self.center = transforms.CenterCrop(224)
        self.to_tensor = transforms.ToTensor()
        self.norm = transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)

    def __len__(self): return len(self.base)

    def __getitem__(self, idx):
        path, y = self.base.samples[idx]
        img = default_loader(path)
        img = self.pre(img); img = self.center(img)
        if self.mode == "brightness":
            img = brightness_shift(img, self.factor)
            t = self.to_tensor(img)
        else:
            t = self.to_tensor(img)
            t = add_gaussian_noise(t, self.sigma)
        t = self.norm(t)
        return t, y

def evaluate(model, loader, criterion):
    model.eval()
    tot, correct, loss_sum = 0, 0, 0.0
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            loss = criterion(logits, y)
            pred = logits.argmax(1)
            correct += (pred==y).sum().item()
            loss_sum += loss.item() * x.size(0)
            tot += x.size(0)
    return {"acc": correct/tot, "loss": loss_sum/tot}


In [31]:
#@title 10) Baseline vs Drift runs in W&B + Alert + Summary Table
import wandb

# Reload best model
ckpt = torch.load("./checkpoints/best.pt", map_location=DEVICE)
model.load_state_dict(ckpt["model"], strict=True)



<All keys matched successfully>

In [32]:
# Baseline evaluation (clean val)
baseline_run = wandb.init(project=ARGS.project, entity=ARGS.entity,
                          name="baseline-eval", job_type="evaluation",
                          config={"eval_split":"val_clean"})
base_metrics = evaluate(model, val_loader, criterion)
wandb.log({"val_clean/acc": base_metrics["acc"], "val_clean/loss": base_metrics["loss"]})
baseline_run.finish()
print("Baseline:", base_metrics)

# Drift configs (feel free to tweak strengths)
drift_cfgs = [
    {"name":"drift-brightness-0.35", "mode":"brightness", "factor":0.35, "sigma":None},
    {"name":"drift-noise-0.20",      "mode":"noise",      "factor":None, "sigma":0.20},
]

summary_rows = []
alert_threshold = 0.10  # 10 points absolute drop

for cfg in drift_cfgs:
    run = wandb.init(project=ARGS.project, entity=ARGS.entity,
                     name=cfg["name"], job_type="drift-eval",
                     config={"eval_split":"val_drift", **{k:v for k,v in cfg.items() if k!='name'}})

    if cfg["mode"] == "brightness":
        dset = DriftedValDataset(val_ds, mode="brightness", factor=cfg["factor"])
    else:
        dset = DriftedValDataset(val_ds, mode="noise", sigma=cfg["sigma"])
    dloader = DataLoader(dset, batch_size=ARGS.batch_size, shuffle=False,
                         num_workers=ARGS.num_workers, pin_memory=True)

    m = evaluate(model, dloader, criterion)
    wandb.log({f"{cfg['name']}/acc": m["acc"], f"{cfg['name']}/loss": m["loss"]})
    print(cfg["name"], m)

    drop = base_metrics["acc"] - m["acc"]
    wandb.log({f"{cfg['name']}/acc_drop": drop})

    # Trigger W&B alert if significant
    if drop >= alert_threshold:
        wandb.alert(
            title="Accuracy Drop Detected",
            text=f"{cfg['name']}: accuracy dropped by {drop*100:.1f} pts (baseline {base_metrics['acc']:.3f} -> {m['acc']:.3f})",
            level=wandb.AlertLevel.WARN
        )

    # Collect for W&B table-style summary
    summary_rows.append([cfg["name"], base_metrics["acc"], m["acc"], drop])
    run.finish()

# Optional: log a small comparison table (mirrors monitoring lab style)
table_run = wandb.init(project=ARGS.project, entity=ARGS.entity, name="drift-summary", job_type="report")
tbl = wandb.Table(columns=["run","baseline_acc","drift_acc","acc_drop"], data=summary_rows)
wandb.log({"drift/summary_table": tbl, "drift/threshold": alert_threshold})
table_run.finish()

print("Open W&B → compare baseline-eval with drift-* runs; include screenshots of charts, table, and alert.")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


val_clean/acc,▁
val_clean/loss,▁
val_clean/acc,0.7372
val_clean/loss,1.00445


Baseline: {'acc': 0.7372, 'loss': 1.0044487812042235}


drift-brightness-0.35 {'acc': 0.5888, 'loss': 1.6894900268554687}


drift-brightness-0.35/acc,▁
drift-brightness-0.35/acc_drop,▁
drift-brightness-0.35/loss,▁
drift-brightness-0.35/acc,0.5888
drift-brightness-0.35/acc_drop,0.1484
drift-brightness-0.35/loss,1.68949


drift-noise-0.20 {'acc': 0.1216, 'loss': 5.259431553649902}


drift-noise-0.20/acc,▁
drift-noise-0.20/acc_drop,▁
drift-noise-0.20/loss,▁
drift-noise-0.20/acc,0.1216
drift-noise-0.20/acc_drop,0.6156
drift-noise-0.20/loss,5.25943


drift/threshold,▁
drift/threshold,0.1


Open W&B → compare baseline-eval with drift-* runs; include screenshots of charts, table, and alert.


In [33]:
# Drift configs (feel free to tweak strengths)
drift_cfgs = [
    {"name":"drift-brightness-0.35", "mode":"brightness", "factor":0.35, "sigma":None},
    {"name":"drift-noise-0.20",      "mode":"noise",      "factor":None, "sigma":0.20},
]

summary_rows = []
alert_threshold = 0.10  # 10 points absolute drop

for cfg in drift_cfgs:
    run = wandb.init(project=ARGS.project, entity=ARGS.entity,
                     name=cfg["name"], job_type="drift-eval",
                     config={"eval_split":"val_drift", **{k:v for k,v in cfg.items() if k!='name'}})

    if cfg["mode"] == "brightness":
        dset = DriftedValDataset(val_ds, mode="brightness", factor=cfg["factor"])
    else:
        dset = DriftedValDataset(val_ds, mode="noise", sigma=cfg["sigma"])
    dloader = DataLoader(dset, batch_size=ARGS.batch_size, shuffle=False,
                         num_workers=ARGS.num_workers, pin_memory=True)

    m = evaluate(model, dloader, criterion)
    wandb.log({f"{cfg['name']}/acc": m["acc"], f"{cfg['name']}/loss": m["loss"]})
    print(cfg["name"], m)

    drop = base_metrics["acc"] - m["acc"]
    wandb.log({f"{cfg['name']}/acc_drop": drop})

    # Trigger W&B alert if significant
    if drop >= alert_threshold:
        wandb.alert(
            title="Accuracy Drop Detected",
            text=f"{cfg['name']}: accuracy dropped by {drop*100:.1f} pts (baseline {base_metrics['acc']:.3f} -> {m['acc']:.3f})",
            level=wandb.AlertLevel.WARN
        )

    # Collect for W&B table-style summary
    summary_rows.append([cfg["name"], base_metrics["acc"], m["acc"], drop])
    run.finish()



drift-brightness-0.35 {'acc': 0.5888, 'loss': 1.6894900268554687}


drift-brightness-0.35/acc,▁
drift-brightness-0.35/acc_drop,▁
drift-brightness-0.35/loss,▁
drift-brightness-0.35/acc,0.5888
drift-brightness-0.35/acc_drop,0.1484
drift-brightness-0.35/loss,1.68949


drift-noise-0.20 {'acc': 0.1233, 'loss': 5.2630565536499025}


drift-noise-0.20/acc,▁
drift-noise-0.20/acc_drop,▁
drift-noise-0.20/loss,▁
drift-noise-0.20/acc,0.1233
drift-noise-0.20/acc_drop,0.6139
drift-noise-0.20/loss,5.26306


In [34]:
# Optional: log a small comparison table (mirrors monitoring lab style)
table_run = wandb.init(project=ARGS.project, entity=ARGS.entity, name="drift-summary", job_type="report")
tbl = wandb.Table(columns=["run","baseline_acc","drift_acc","acc_drop"], data=summary_rows)
wandb.log({"drift/summary_table": tbl, "drift/threshold": alert_threshold})
table_run.finish()

print("Open W&B → compare baseline-eval with drift-* runs; include screenshots of charts, table, and alert.")

drift/threshold,▁
drift/threshold,0.1


Open W&B → compare baseline-eval with drift-* runs; include screenshots of charts, table, and alert.
